# Create the retrieval part of the RAG 

More specifically we will:
- Read a JSON document containing the internal information that we need
- Connect and create an index in the elastic search
- Index the document using elastic-search, to retrieve it later creating a small search engine

In [ ]:
# Download the document containing our own knowledge base
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
# Examine the first record of the file
!head documents.json

In [ ]:
# Load and flatten the file by adding the course in each object in the question list

# Import the necessary libraries to read JSON files
import json

# Read the JSON file
with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

# Initialise the flattened list of qas
documents = []

# Flatten the file
for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
# See the number of questions we have
print(len(documents))
# See the first q&a
documents[0]

In [ ]:
from elasticsearch import Elasticsearch
# Connect to the elastic search
es = Elasticsearch("http://localhost:9200")
# Verify that you have connected successfully 
es.info()

In [ ]:
# Provide the properties of the elastic search index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
# Provide the name of the index
index_name = "course-questions"
# Create the Index 
response = es.indices.create(index=index_name, body=index_settings)
# Verify that the index has been created
response

In [ ]:
from tqdm.auto import tqdm
# Index all the document to elastic search - adding document to the specific index
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

In [ ]:
# Simple query data for an elastic search

# Create the question for the elastic search to look for
user_question = "How do I join the course after it has started?"

# Create the body of the search request
search_query = {
    "size": 5,     # specify the number of documents to retrieve
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,  # specify the question to elastic search
                    "fields": ["question^3", "text", "section"], # specify the field you want the elastic search to look for answers - the ^3 meaning that we will give 3 times more priority to answers found in the question field
                    "type": "best_fields"
                }
            },
            "filter": {   # to filter from which document to look into
                "term": {
                    "course": "data-engineering-zoomcamp" 
                }
            }
        }
    }
}

# Query the elactic search db
response = es.search(index=index_name, body=search_query)

# See the response -  top 5 search results
response

In [ ]:
# Prettify the response
for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

In [ ]:
# To have them all together in one function

# Initialize the elastic search via docker
es = Elasticsearch("http://localhost:9200")

# Create the function to query the user question in Elastic Search
def retrieve_documents(query, index_name="course-questions", max_results=5):    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [ ]:
# Query a question
user_question = "How do I join the course after it has started?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

# Create the answering generation part of the RAG 

More specifically we will:
- Create a client of Open AI API
- Connect a prompt template containing the model instructions, question, and context (to instruct the model to answer based on the context)
- Create the context in the prompt by appending there all the retrieved documents from elastic search
- Use this prompt to the API to get the answer

In [ ]:
from openai import OpenAI

# Create a client to the OpenAI API
client = OpenAI()

# Verify the connection by providing a sample question
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "What's the formula for Energy?"}]
)
print(response.choices[0].message.content)

In [ ]:
# Create the context of the prompt

# Retrieve the relevant information from elastic search
context_docs = retrieve_documents(user_question)

# Create the context string
context = ""

for doc in context_docs:
    doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    context += doc_str

context = context.strip()
print(context)

In [ ]:
# Create the final prompt containing instructions, question and context
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contain the answer, return "NONE"

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [ ]:
# Use this prompt to the API
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)
answer = response.choices[0].message.content
answer

In [ ]:
# Create a function to perform the whole RAG

# Create a function to formulate the context string
def build_context(documents):
    context = ""

    for doc in documents:
        doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
        context += doc_str
    
    context = context.strip()
    return context

# Create a function to formulate the final prompt
def build_prompt(user_question, documents):
    context = build_context(documents)
    return f"""
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

# Create a function to sent the request to LLM model using the constructed prompt
def ask_openai(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer

# Create the QA bot using RAG
def qa_bot(user_question):
    # Retrieve the relevant document using elastic search
    context_docs = retrieve_documents(user_question)
    # Create the prompt
    prompt = build_prompt(user_question, context_docs)
    # Pass the prompt to the LLM model aka OpenAI API
    answer = ask_openai(prompt)
    return answer

In [ ]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")